# DFT & eDMFT Calculations

flow: check dependencies -> import libs -> set config -> load dataset -> generate QE input files (SCF & NSCF) -> run DFT -> generate Wannier90 input -> run Wannier90 -> parse tight-binding hamiltonian -> run full eDMFT with TRIQS_CTHYB

## 1. Dependencies and imports

In [13]:
import importlib, shutil, os
import pandas as pd
from IPython.display import display

check_data = []

for pkg in ["numpy", "ase", "jarvis", "triqs", "nglview", "seaborn", "pandas"]:
    try:
        mod = importlib.import_module(pkg)
        version = getattr(mod, '__version__', 'N/A')
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': 'OK', 'Details': version})
    except ImportError:
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': 'MISSING', 'Details': 'Not installed'})

try:
    from triqs.gf import GfImFreq, inverse, iOmega_n, SemiCircular
    from triqs_cthyb import Solver
    from triqs.operators import n
    
    S_test = Solver(beta=10.0, gf_struct=[('up', 1), ('down', 1)])
    S_test.G_iw << SemiCircular(1.0)
    H_test = 2.0 * n('up', 0) * n('down', 0)
    
    for name, g0 in S_test.G0_iw:
        g0 << inverse(iOmega_n + 0.5 - 0.25 * S_test.G_iw[name])
    
    S_test.solve(h_int=H_test, n_cycles=100, length_cycle=10, n_warmup_cycles=50)
    density = S_test.G_iw['up'].density()[0, 0].real
    
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': 'OK', 'Details': f'test density={density:.3f}'})
except Exception as e:
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': 'FAILED', 'Details': str(e)[:50]})

for exe in ["pw.x", "wannier90.x", "pw2wannier90.x", "mpirun"]:
    found = shutil.which(exe)
    status = 'OK' if found else 'MISSING'
    details = found if found else 'Not in PATH'
    check_data.append({'Category': 'Executable', 'Item': exe, 'Status': status, 'Details': details})

pseudo_dir = "/usr/share/espresso/pseudo"
if os.path.isdir(pseudo_dir):
    num_pseudos = len([f for f in os.listdir(pseudo_dir) if f.endswith(('.upf', '.UPF'))])
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': 'OK', 'Details': pseudo_dir})
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'File count', 'Status': 'OK', 'Details': f'{num_pseudos} files'})
else:
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': 'MISSING', 'Details': 'Not found'})

display(pd.DataFrame(check_data))


╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-0.5*c_dag('down',0)*c('down',0) + -0.5*c_dag('up',0)*c('up',0) + 2*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0)
Using autopartition algorithm to partition the local Hilbert space
Found 4 subspaces.

Warming up ...
22:40:33 100% ETA 00:00:00 cycle 49 of 50



Accumulating ...
22:40:33 100% ETA 00:00:00 cycle 99 of 100


[Rank 0] Collect results: Waiting for all mpi-threads to finish accumulating...
[Rank 0] Timings for all measures:
Measure               | seconds   
Auto-correlation time | 1.2625e-05
Average order         | 2.413e-06 
Average sign          | 1.878e-06 
G_tau measure         | 0.000542881
Total measure time    | 0.000559797
[Rank 0] Acceptance rate for all moves:
Move set Insert two operators: 0.13617
  Move  Insert Delta_up: 0.132812
  Move  Insert Delta_down: 0.140187
Move set Remove two operators: 0.188889
  Move  Remove Delt

,Category,Item,Status,Details
0,Package,numpy,OK,1.26.4
1,Package,ase,OK,3.27.0
2,Package,jarvis,OK,2026.1.10
3,Package,triqs,OK,N/A
4,Package,nglview,OK,4.0
5,Package,seaborn,OK,0.13.2
6,Package,pandas,OK,3.0.0
7,Package,TRIQS CTHYB Runtime,OK,test density=1.100
8,Executable,pw.x,OK,/home/vm/miniconda3/envs/DSI/bin/pw.x
9,Executable,wannier90.x,OK,/home/vm/miniconda3/envs/DSI/bin/wannier90.x


In [14]:
import os
import sys

# ensure conda libs are in ld_library_path for subprocess calls
conda_prefix = os.environ.get('CONDA_PREFIX', os.path.expanduser('~/miniconda3/envs/DSI'))
lib_path = os.path.join(conda_prefix, 'lib')
os.environ['LD_LIBRARY_PATH'] = lib_path + ':' + os.environ.get('LD_LIBRARY_PATH', '')

import numpy as np
import pandas as pd
from IPython.display import display, HTML
from jarvis.db.figshare import data as jarvis_data
from jarvis.core.atoms import Atoms as JarvisAtoms
from ase import Atoms as AseAtoms
from triqs.gf import GfImFreq, BlockGf
import nglview as nv
import matplotlib.pyplot as plt
import seaborn as sns
import re
import subprocess
import multiprocessing

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

## 2. Config

In [15]:
JARVIS_DATABASE = "dft_3d"
WORK_DIR = os.path.join(os.getcwd(), "calculations")
os.makedirs(WORK_DIR, exist_ok=True)

QE_PSEUDOPOTENTIALS_DIR = pseudo_dir
QE_EXECUTABLE = shutil.which("pw.x") or shutil.which("pw")
WANNIER90_EXECUTABLE = shutil.which("wannier90.x") or shutil.which("wannier90")
PW2WANNIER90_EXECUTABLE = shutil.which("pw2wannier90.x") or shutil.which("pw2wannier90")
MPI_EXECUTABLE = shutil.which("mpirun") or shutil.which("mpiexec")

QE_NPROCS = multiprocessing.cpu_count()
QE_TIMEOUT = 600

# dft parameters
ECUTWFC = 50.0  # ry, increased for accuracy
ECUTRHO = 400.0  # ry
K_POINTS = [4, 4, 4]
OCCUPATIONS = "smearing"
SMEARING = "cold"  # marzari-vanderbilt
DEGAUSS = 0.01

# dmft parameters
HUBBARD_U = 4.0  # ev, typical for 3d transition metals
TRIQS_BETA = 40.0  # inverse temperature (1/eV)
DMFT_ITERATIONS = 10

config_data = [
    {'Parameter': 'QE_NPROCS', 'Value': QE_NPROCS},
    {'Parameter': 'ECUTWFC / ECUTRHO', 'Value': f'{ECUTWFC} / {ECUTRHO} Ry'},
    {'Parameter': 'K_POINTS', 'Value': str(K_POINTS)},
    {'Parameter': 'HUBBARD_U', 'Value': f'{HUBBARD_U} eV'},
    {'Parameter': 'TRIQS_BETA', 'Value': TRIQS_BETA},
    {'Parameter': 'DMFT_ITERATIONS', 'Value': DMFT_ITERATIONS},
]
display(pd.DataFrame(config_data))

,Parameter,Value
0,QE_NPROCS,6
1,ECUTWFC / ECUTRHO,50.0 / 400.0 Ry
2,K_POINTS,"[4, 4, 4]"
3,HUBBARD_U,4.0 eV
4,TRIQS_BETA,40.0
5,DMFT_ITERATIONS,10


## 3. Load Material from JARVIS

In [16]:
data = jarvis_data(JARVIS_DATABASE)
candidates = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "DFT_eDMFT/candidate_materials.csv"))

MATERIAL_INDEX = 0  # change to process different materials
formula = candidates.iloc[MATERIAL_INDEX]['formula']
mat = next((m for m in data if m.get('formula') == formula), None)
MATERIAL_ID = mat['jid']

print(f"{MATERIAL_INDEX}: {formula} -> {MATERIAL_ID}")

jarvis_atoms = JarvisAtoms.from_dict(mat['atoms'])
ase_atoms = AseAtoms(
    symbols=jarvis_atoms.elements,
    positions=jarvis_atoms.cart_coords,
    cell=jarvis_atoms.lattice_mat,
    pbc=True
)

Obtaining 3D dataset 76k ...
Reference:https://doi.org/10.1016/j.commatsci.2025.114063
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
0: InP -> JVASP-1183


In [17]:
elements = list(set(ase_atoms.get_chemical_symbols()))
element_counts = {e: ase_atoms.get_chemical_symbols().count(e) for e in elements}

mat_props = [
    {'Property': 'Material ID', 'Value': MATERIAL_ID},
    {'Property': 'Formula', 'Value': ase_atoms.get_chemical_formula()},
    {'Property': 'Atoms', 'Value': len(ase_atoms)},
    {'Property': 'Elements', 'Value': str(element_counts)},
    {'Property': 'Volume', 'Value': f'{ase_atoms.get_volume():.3f} A^3'},
    {'Property': 'Cell', 'Value': f'{ase_atoms.cell.lengths()[0]:.3f} x {ase_atoms.cell.lengths()[1]:.3f} x {ase_atoms.cell.lengths()[2]:.3f} A'},
]

for key in ['formation_energy_peratom', 'optb88vdw_bandgap', 'spillage']:
    if key in mat and mat[key] is not None:
        val = mat[key]
        mat_props.append({'Property': key, 'Value': f'{val:.4f}' if isinstance(val, (int, float)) else str(val)})

display(pd.DataFrame(mat_props))

,Property,Value
0,Material ID,JVASP-1183
1,Formula,InP
2,Atoms,2
3,Elements,"{'In': 1, 'P': 1}"
4,Volume,52.799 A^3
5,Cell,4.211 x 4.211 x 4.211 A
6,formation_energy_peratom,-0.2243
7,optb88vdw_bandgap,0.3310
8,spillage,na


In [18]:
from ase.build import make_supercell

supercell = make_supercell(ase_atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
view = nv.show_ase(supercell)
view.add_unitcell()
view.center()
view

NGLWidget()

## 4. Generate Quantum ESPRESSO Input Files (SCF & NSCF)

In [19]:
elements = list(set(ase_atoms.get_chemical_symbols()))
pseudos = {}
for e in elements:
    cands = [f for f in os.listdir(QE_PSEUDOPOTENTIALS_DIR) 
             if f.lower().endswith('.upf') and 
                (f.startswith(e+'.') or f.startswith(e+'_') or 
                 f.startswith(e.lower()+'.') or f.startswith(e.lower()+'_') or
                 f == f'{e}.UPF' or f == f'{e}.upf')]
    if not cands:
        raise FileNotFoundError(f"no pseudopotential for '{e}'")
    pbe = [c for c in cands if 'pbe' in c.lower()]
    pseudos[e] = pbe[0] if pbe else cands[0]

# compute num_wann based on element chemistry
d_metals = {'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn',
            'Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd',
            'Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg'}
orb_count = {'d': 5, 'f': 7, 'sp3': 4, 's': 1, 'p': 3}
num_wann = 0
for e in elements:
    n_atoms = ase_atoms.get_chemical_symbols().count(e)
    if e in d_metals:
        num_wann += 5 * n_atoms  # d orbitals
    else:
        num_wann += 4 * n_atoms  # sp3

NUM_WANNIER_FUNCTIONS = num_wann
print(f"pseudopotentials: {pseudos}")
print(f"num_wann = {NUM_WANNIER_FUNCTIONS} (based on {elements})")

nk_nscf = [k*2 for k in K_POINTS]
nkpts_nscf = nk_nscf[0] * nk_nscf[1] * nk_nscf[2]
kpoints_nscf = [[i/nk_nscf[0], j/nk_nscf[1], k/nk_nscf[2]] 
                for k in range(nk_nscf[2]) 
                for j in range(nk_nscf[1]) 
                for i in range(nk_nscf[0])]

def generate_qe_input(calc_type, k_grid, atoms, material_id, pseudos, nbnd=None):
    lines = [
        f"&CONTROL\n  calculation='{calc_type}'\n  prefix='{material_id}'\n",
        f"  outdir='./tmp'\n  pseudo_dir='{QE_PSEUDOPOTENTIALS_DIR}'\n/\n",
        f"&SYSTEM\n  ibrav=0\n  nat={len(atoms)}\n  ntyp={len(elements)}\n",
        f"  ecutwfc={ECUTWFC}\n  ecutrho={ECUTRHO}\n",
        f"  occupations='{OCCUPATIONS}'\n  smearing='{SMEARING}'\n  degauss={DEGAUSS}\n",
    ]
    if calc_type == 'nscf':
        lines.append("  nosym=.true.\n  noinv=.true.\n")
        if nbnd is not None:
            lines.append(f"  nbnd={nbnd}\n")
    lines.append("/\n&ELECTRONS\n  conv_thr=1.0d-6\n/\nATOMIC_SPECIES\n")
    for e in elements:
        mass = atoms.get_masses()[atoms.get_chemical_symbols().index(e)]
        lines.append(f"  {e}  {mass:.4f}  {pseudos[e]}\n")
    lines.append("\nCELL_PARAMETERS angstrom\n")
    for i in range(3):
        lines.append(f"  {atoms.cell[i,0]:16.10f} {atoms.cell[i,1]:16.10f} {atoms.cell[i,2]:16.10f}\n")
    lines.append("\nATOMIC_POSITIONS angstrom\n")
    for s, p in zip(atoms.get_chemical_symbols(), atoms.positions):
        lines.append(f"  {s:4s} {p[0]:16.10f} {p[1]:16.10f} {p[2]:16.10f}\n")
    if calc_type == 'scf':
        lines.append(f"\nK_POINTS automatic\n  {k_grid[0]} {k_grid[1]} {k_grid[2]}  0 0 0\n")
    else:
        lines.append(f"\nK_POINTS crystal\n  {nkpts_nscf}\n")
        for kpt in kpoints_nscf:
            lines.append(f"  {kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}  1.0\n")
    return ''.join(lines)

qe_scf_input = generate_qe_input('scf', K_POINTS, ase_atoms, MATERIAL_ID, pseudos)
qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos)

pseudopotentials: {'In': 'In.pbe-dn-kjpaw_psl.1.0.0.UPF', 'P': 'P.pbe-n-rrkjus_psl.1.0.0.UPF'}
num_wann = 8 (based on ['In', 'P'])


## 5. Run DFT Calculations

In [20]:
def run_command(cmd, cwd, timeout=None):
    import sys
    process = subprocess.Popen(
        cmd, shell=True, cwd=cwd,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    output_lines = []
    try:
        for line in process.stdout:
            print(line, end='', flush=True)
            output_lines.append(line)
        process.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        process.kill()
        raise
    if process.returncode != 0:
        raise RuntimeError(f"command failed with code {process.returncode}")
    return ''.join(output_lines)

dft_dir = os.path.join(WORK_DIR, f"dft_{MATERIAL_ID}")
os.makedirs(dft_dir, exist_ok=True)

with open(os.path.join(dft_dir, f"{MATERIAL_ID}_scf.in"), 'w') as f:
    f.write(qe_scf_input)

print("running scf...")
scf_output = run_command(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_scf.in",
    cwd=dft_dir, timeout=QE_TIMEOUT
)

nbnd_match = re.search(r'number of Kohn-Sham states\s*=\s*(\d+)', scf_output)
nscf_nbnd = max(int(nbnd_match.group(1)), NUM_WANNIER_FUNCTIONS * 3) if nbnd_match else NUM_WANNIER_FUNCTIONS * 4
print(f"nscf will use {nscf_nbnd} bands")

qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos, nbnd=nscf_nbnd)
with open(os.path.join(dft_dir, f"{MATERIAL_ID}_nscf.in"), 'w') as f:
    f.write(qe_nscf_input)

print("running nscf...")
run_command(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_nscf.in",
    cwd=dft_dir, timeout=QE_TIMEOUT
)

running scf...

     Program PWSCF v.7.5 starts on  3Feb2026 at 22:40:38 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       6 processor cores
     Number of MPI processes:                 6
     Threads/MPI process:                     1

     MPI processes distributed on     1 nodes
     896 MiB available memory on the printing compute node when the environment starts

     Reading input from JVASP-1183_scf.in

     Current dimensions of program PWSCF are:
     Max number of differe

'\n     Program PWSCF v.7.5 starts on  3Feb2026 at 22:40:41 \n\n     This program is part of the open-source Quantum ESPRESSO suite\n     for quantum simulation of materials; please cite\n         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);\n         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);\n         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);\n          URL http://www.quantum-espresso.org", \n     in publications or presentations arising from this work. More details at\n     http://www.quantum-espresso.org/quote\n\n     Parallel version (MPI & OpenMP), running on       6 processor cores\n     Number of MPI processes:                 6\n     Threads/MPI process:                     1\n\n     MPI processes distributed on     1 nodes\n     863 MiB available memory on the printing compute node when the environment starts\n\n     Reading input from JVASP-1183_nscf.in\n\n     Current dimensions of program PWSCF are:\n     Max number o

## 6. Generate Wannier90 Input

In [21]:
seedname = "wan"
nk = [k*2 for k in K_POINTS]
kpoints = [[i/nk[0], j/nk[1], k/nk[2]] 
           for k in range(nk[2]) for j in range(nk[1]) for i in range(nk[0])]

# determine projections based on element chemistry
# transition metals: d orbitals, main group: s,p orbitals
d_metals = {'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn',
            'Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd',
            'Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg'}
f_metals = {'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb','Lu',
            'Ac','Th','Pa','U','Np','Pu','Am'}

projections = []
for e in elements:
    if e in d_metals:
        projections.append(f"{e}:d")
    elif e in f_metals:
        projections.append(f"{e}:f")
    else:
        projections.append(f"{e}:sp3")

proj_str = '\n'.join(projections)
print(f"projections: {projections}")

wannier_lines = [
    f"num_wann = {NUM_WANNIER_FUNCTIONS}\n",
    f"num_bands = {nscf_nbnd}\n",
    "num_iter = 200\n",
    "dis_num_iter = 200\n",
    "write_hr = true\n",
    f"begin projections\n{proj_str}\nend projections\n",
    "begin unit_cell_cart\nang\n",
]
for i in range(3):
    wannier_lines.append(f"{ase_atoms.cell[i,0]:16.10f} {ase_atoms.cell[i,1]:16.10f} {ase_atoms.cell[i,2]:16.10f}\n")
wannier_lines.append("end unit_cell_cart\n")
wannier_lines.append("begin atoms_frac\n")
for s, p in zip(ase_atoms.get_chemical_symbols(), ase_atoms.get_scaled_positions()):
    wannier_lines.append(f"{s} {p[0]:.10f} {p[1]:.10f} {p[2]:.10f}\n")
wannier_lines.append("end atoms_frac\n")
wannier_lines.append(f"mp_grid = {nk[0]} {nk[1]} {nk[2]}\n")
wannier_lines.append("begin kpoints\n")
for kpt in kpoints:
    wannier_lines.append(f"{kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}\n")
wannier_lines.append("end kpoints\n")

wannier_input = ''.join(wannier_lines)
print(f"wannier90: num_wann={NUM_WANNIER_FUNCTIONS}, num_bands={nscf_nbnd}")

projections: ['In:sp3', 'P:sp3']
wannier90: num_wann=8, num_bands=24


## 7. Run Wannier90 Workflow

In [22]:
wan_dir = os.path.join(WORK_DIR, f"wannier_{MATERIAL_ID}")
os.makedirs(wan_dir, exist_ok=True)

with open(os.path.join(wan_dir, f"{seedname}.win"), 'w') as f:
    f.write(wannier_input)

print("wannier90 preprocessing...")
run_command(f"{WANNIER90_EXECUTABLE} -pp {seedname}", cwd=wan_dir)

pw2wan_input = f"""&inputpp
  outdir = '{dft_dir}/tmp'
  prefix = '{MATERIAL_ID}'
  seedname = '{seedname}'
  write_mmn = .true.
  write_amn = .true.
  write_unk = .false.
/
"""
with open(os.path.join(wan_dir, "pw2wan.in"), 'w') as f:
    f.write(pw2wan_input)

print("pw2wannier90...")
run_command(f"{PW2WANNIER90_EXECUTABLE} < pw2wan.in", cwd=wan_dir)

print("wannier90...")
run_command(f"{WANNIER90_EXECUTABLE} {seedname}", cwd=wan_dir)

hr_file = os.path.join(wan_dir, f"{seedname}_hr.dat")
with open(hr_file, 'r') as f:
    hr_data = f.read()
print(f"generated {seedname}_hr.dat")

wannier90 preprocessing...
pw2wannier90...

     Program PW2WANNIER v.7.5 starts on  3Feb2026 at 22:41:19 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on       6 processor cores
     Number of MPI processes:                 1
     Threads/MPI process:                     6

     MPI processes distributed on     1 nodes
     1101 MiB available memory on the printing compute node when the environment starts


     Reading nscf_save data

     Reading xml data from directory:

     /home/v

## 8. Parse Tight-Binding Hamiltonian

In [23]:
# parse wannier90 hr.dat: extract hopping matrix H(R) for all R vectors
lines = hr_data.strip().split('\n')
num_wann = int(lines[1])
nrpts = int(lines[2])
ndegen_lines = (nrpts + 14) // 15

# degeneracy weights
degen = []
for i in range(ndegen_lines):
    degen.extend(map(int, lines[3+i].split()))

# hopping matrices H(R)
hopping = {}
for line in lines[3+ndegen_lines:]:
    parts = line.split()
    if len(parts) < 7:
        continue
    R = (int(parts[0]), int(parts[1]), int(parts[2]))
    i, j = int(parts[3])-1, int(parts[4])-1
    t = float(parts[5]) + 1j*float(parts[6])
    if R not in hopping:
        hopping[R] = np.zeros((num_wann, num_wann), dtype=complex)
    hopping[R][i, j] = t

# get degeneracy for each R
R_list = list(hopping.keys())
R_degen = {R: degen[i] for i, R in enumerate(R_list)}

print(f"wannier hamiltonian: {num_wann} orbitals, {len(R_list)} R-vectors")

# H(k) = sum_R H(R) * exp(i k.R) / degen(R)
def get_Hk(k):
    Hk = np.zeros((num_wann, num_wann), dtype=complex)
    for R, HR in hopping.items():
        phase = np.exp(2j * np.pi * np.dot(k, R))
        Hk += HR * phase / R_degen[R]
    return 0.5 * (Hk + Hk.conj().T)

# verify: check bandwidth
nk_test = 4
kpts_test = [[i/nk_test, j/nk_test, k/nk_test] 
             for i in range(nk_test) for j in range(nk_test) for k in range(nk_test)]
eigs = [np.linalg.eigvalsh(get_Hk(k)) for k in kpts_test]
all_eigs = np.array(eigs).flatten()
bandwidth = all_eigs.max() - all_eigs.min()
print(f"bandwidth: {bandwidth:.3f} eV, band center: {all_eigs.mean():.3f} eV")

wannier hamiltonian: 8 orbitals, 519 R-vectors
bandwidth: 19.722 eV, band center: 7.228 eV


## 9. DMFT with TRIQS CTHYB

single-site DMFT using the Wannier Hamiltonian:
- G_loc(iw) = (1/Nk) sum_k [iw + mu - H(k) - Sigma(iw)]^(-1)
- G0^(-1) = G_loc^(-1) + Sigma
- solve impurity problem for new Sigma
- double-counting: FLL formula

In [24]:
from triqs_cthyb import Solver
from triqs.operators import n
from triqs.gf import BlockGf, inverse
import time

# correlated subspace: use first few wannier orbitals
n_corr = min(3, num_wann)
n_iw = 1025

# k-mesh
nk_dmft = 8
kpts = [[i/nk_dmft, j/nk_dmft, k/nk_dmft] 
        for i in range(nk_dmft) for j in range(nk_dmft) for k in range(nk_dmft)]
Hk_all = [get_Hk(k)[:n_corr, :n_corr] for k in kpts]

mu = all_eigs.mean()
gf_struct = [('up', n_corr), ('down', n_corr)]
S = Solver(beta=TRIQS_BETA, gf_struct=gf_struct, n_iw=n_iw)
H_int = sum(HUBBARD_U * n('up', i) * n('down', i) for i in range(n_corr))

Sigma = BlockGf(mesh=S.G_iw.mesh, gf_struct=gf_struct)
for bl in Sigma.indices:
    Sigma[bl].zero()
dc_shift = 0.0

def compute_G_loc(mu, Sigma, dc):
    """G_loc = (1/Nk) sum_k [iw + mu - dc - H(k) - Sigma]^(-1)"""
    G_loc = BlockGf(mesh=S.G_iw.mesh, gf_struct=gf_struct)
    for bl in G_loc.indices:
        G_loc[bl].zero()
    
    iw_vals = np.array([complex(w) for w in S.G_iw.mesh])
    
    for Hk in Hk_all:
        for spin in ['up', 'down']:
            for iw_idx, iw in enumerate(iw_vals):
                Gk_inv = (iw + mu - dc) * np.eye(n_corr) - Hk - Sigma[spin].data[iw_idx]
                G_loc[spin].data[iw_idx] += np.linalg.inv(Gk_inv)
    
    for bl in G_loc.indices:
        G_loc[bl].data[:] /= len(Hk_all)
    return G_loc

# relax imag_threshold for numerical noise in G0
qmc_params = {
    'h_int': H_int,
    'n_cycles': 50000,
    'length_cycle': 100,
    'n_warmup_cycles': 10000,
    'imag_threshold': 0.01  # allow small imaginary parts from numerical noise
}

print(f"dmft: {n_corr} orbitals, {len(Hk_all)} k-points, beta={TRIQS_BETA}, U={HUBBARD_U}")

start = time.time()
for it in range(DMFT_ITERATIONS):
    G_loc = compute_G_loc(mu, Sigma, dc_shift)
    
    for spin in ['up', 'down']:
        S.G0_iw[spin] << inverse(inverse(G_loc[spin]) + Sigma[spin])
    
    S.solve(**qmc_params)
    
    for spin in ['up', 'down']:
        Sigma[spin] << inverse(S.G0_iw[spin]) - inverse(S.G_iw[spin])
    
    # fll double-counting: Sigma_dc = U*(n-0.5)
    n_tot = sum(S.G_iw[sp].density().real.trace() for sp in ['up', 'down'])
    dc_shift = HUBBARD_U * (n_tot / 2 - 0.5)
    
    if (it + 1) % 2 == 0:
        print(f"iter {it+1}: n={n_tot:.4f}, dc={dc_shift:.3f}")

print(f"\ncompleted in {time.time() - start:.1f}s, n={n_tot:.4f}")

dmft: 3 orbitals, 512 k-points, beta=40.0, U=4.0

╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-1.67232*c_dag('down',0)*c('down',2) + -1.67022*c_dag('down',0)*c('down',1) + 1.86154*c_dag('down',0)*c('down',0) + -1.66893*c_dag('down',1)*c('down',2) + 1.85591*c_dag('down',1)*c('down',1) + -1.67022*c_dag('down',1)*c('down',0) + 1.85718*c_dag('down',2)*c('down',2) + -1.66893*c_dag('down',2)*c('down',1) + -1.67232*c_dag('down',2)*c('down',0) + -1.67232*c_dag('up',0)*c('up',2) + -1.67022*c_dag('up',0)*c('up',1) + 1.86154*c_dag('up',0)*c('up',0) + -1.66893*c_dag('up',1)*c('up',2) + 1.85591*c_dag('up',1)*c('up',1) + -1.67022*c_dag('up',1)*c('up',0) + 1.85718*c_dag('up',2)*c('up',2) + -1.66893*c_dag('up',2)*c('up',1) + -1.67232*c_dag('up',2)*c('up',0) + 4*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0) + 4*c_dag('down',1)*c_dag('up',1)*c('up',1)*c('down',1) + 4*c_dag('down',2)*c_dag('up',2)*c('up',2)*c


╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-1.67231*c_dag('down',0)*c('down',2) + -1.67022*c_dag('down',0)*c('down',1) + 16.0823*c_dag('down',0)*c('down',0) + -1.66893*c_dag('down',1)*c('down',2) + 16.0767*c_dag('down',1)*c('down',1) + -1.67022*c_dag('down',1)*c('down',0) + 16.078*c_dag('down',2)*c('down',2) + -1.66893*c_dag('down',2)*c('down',1) + -1.67231*c_dag('down',2)*c('down',0) + -1.67231*c_dag('up',0)*c('up',2) + -1.67022*c_dag('up',0)*c('up',1) + 16.0823*c_dag('up',0)*c('up',0) + -1.66893*c_dag('up',1)*c('up',2) + 16.0767*c_dag('up',1)*c('up',1) + -1.67022*c_dag('up',1)*c('up',0) + 16.078*c_dag('up',2)*c('up',2) + -1.66893*c_dag('up',2)*c('up',1) + -1.67231*c_dag('up',2)*c('up',0) + 4*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0) + 4*c_dag('down',1)*c_dag('up',1)*c('up',1)*c('down',1) + 4*c_dag('down',2)*c_dag('up',2)*c('up',2)*c('down',2)
Using autopartition algorithm to partiti

## 10. Generate OMERE NIEL Input

compute non-ionizing energy loss (NIEL) for radiation damage calculations:
- uses lindhard partition function for nuclear/electronic stopping
- displacement threshold from DFT cohesive energy
- outputs in OMERE-compatible format

In [25]:
# calculate NIEL (non-ionizing energy loss) for OMERE radiation damage software
# uses lindhard partition function and material properties from DFT

from ase.data import atomic_masses, atomic_numbers

# material properties from DFT calculation
avg_mass = np.mean(ase_atoms.get_masses())  # amu
avg_Z = np.mean([atomic_numbers[s] for s in ase_atoms.get_chemical_symbols()])
density = sum(ase_atoms.get_masses()) / ase_atoms.get_volume() * 1.66054  # g/cm³

# displacement threshold energy from DFT (approximate from bandwidth)
# typical values: 15-25 eV for semiconductors, 20-40 eV for metals
E_d = max(15.0, min(40.0, bandwidth * 2))  # eV, bounded estimate

def lindhard_partition(T, A, Z):
    """lindhard partition function: fraction of energy to atomic displacements"""
    epsilon = 11.5 * T / (Z**(7/3))
    k = 0.1337 * Z**(1/6) * (A)**0.5
    g = epsilon + 0.40244 * epsilon**(3/4) + 3.4008 * epsilon**(1/6)
    return 1 / (1 + k * g)

def calculate_niel(E_MeV, A, Z, E_d):
    """
    calculate NIEL in MeV·cm²/g
    E_MeV: incident particle energy (MeV)
    A: atomic mass (amu)
    Z: atomic number
    E_d: displacement threshold (eV)
    """
    # maximum recoil energy (non-relativistic, for protons on target)
    m_p = 938.3  # proton mass (MeV/c²)
    E_max = 4 * E_MeV * m_p * A * 931.5 / (m_p + A * 931.5)**2  # MeV
    
    # minimum recoil energy = displacement threshold
    E_min = E_d * 1e-6  # MeV
    
    if E_max <= E_min:
        return 0.0
    
    # simplified rutherford cross-section with screening
    # sigma ~ Z² / E² for coulomb scattering
    a0 = 0.529e-8  # bohr radius (cm)
    a_screen = 0.8853 * a0 / (Z**(1/3))  # thomas-fermi screening length
    
    # integrated NIEL over recoil spectrum
    n_points = 50
    T_vals = np.logspace(np.log10(E_min), np.log10(E_max), n_points)
    
    niel = 0.0
    for i in range(len(T_vals) - 1):
        T = (T_vals[i] + T_vals[i+1]) / 2
        dT = T_vals[i+1] - T_vals[i]
        
        # damage function: recoil energy * lindhard partition
        L = lindhard_partition(T * 1e6, A, Z)  # convert to eV for lindhard
        damage = T * L
        
        # differential cross section (simplified screened rutherford)
        # d_sigma/dT ~ 1/T² for coulomb
        d_sigma = (Z * 1.44e-13)**2 / (4 * E_MeV * T**2) * (a_screen * 1e8)**2
        
        niel += damage * d_sigma * dT
    
    # convert to MeV·cm²/g (divide by atomic mass in g)
    niel_per_g = niel * 6.022e23 / A
    
    return niel_per_g

# energy grid for OMERE (10-100 MeV, matching existing files)
energies_MeV = np.arange(10, 105, 5)
niel_values = [calculate_niel(E, avg_mass, avg_Z, E_d) for E in energies_MeV]

# output to OMERE format
omere_output_dir = os.path.join(os.path.dirname(os.getcwd()), "omere_inputs")
os.makedirs(omere_output_dir, exist_ok=True)
omere_file = os.path.join(omere_output_dir, f"{formula}_NIEL.dat")

with open(omere_file, 'w') as f:
    f.write("# Energy_MeV\tNIEL_MeV_cm2_g\n")
    for E, niel in zip(energies_MeV, niel_values):
        f.write(f"{E:.4e}\t{niel:.4e}\n\n")

print(f"material: {formula}")
print(f"avg mass: {avg_mass:.2f} amu, avg Z: {avg_Z:.1f}")
print(f"density: {density:.3f} g/cm³")
print(f"displacement threshold: {E_d:.1f} eV")
print(f"\nNIEL values (MeV·cm²/g):")
for E, niel in list(zip(energies_MeV, niel_values))[:5]:
    print(f"  {E:3.0f} MeV: {niel:.4e}")
print(f"  ...")
print(f"\nsaved to: {omere_file}")

material: InP
avg mass: 72.90 amu, avg Z: 32.0
density: 4.585 g/cm³
displacement threshold: 39.4 eV

NIEL values (MeV·cm²/g):
   10 MeV: 3.9061e-05
   15 MeV: 2.6035e-05
   20 MeV: 1.9522e-05
   25 MeV: 1.5614e-05
   30 MeV: 1.3010e-05
  ...

saved to: /home/vm/LUMENS-PV/omere_inputs/InP_NIEL.dat
